In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
import pandas as pd

In [31]:
def init_browser():
    executable_path = {"executable_path":"/usr/local/bin/chromedriver"}
    return Browser("chrome",**executable_path, headless = True)

In [15]:
url = 'https://space-facts.com/mars/'

facts_data = pd.read_html(url)
new_data = []
for frame in facts_data:
    
    new_dict = {}
    for index, row in frame.iterrows():
        new_dict[row[0]] = row[1]
    new_data.append(new_dict)
print(new_data)

[{'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & Deimos)', 'Orbit Distance:': '227,943,824 km (1.38 AU)', 'Orbit Period:': '687 days (1.9 years)', 'Surface Temperature:': '-87 to -5 °C', 'First Record:': '2nd millennium BC', 'Recorded By:': 'Egyptian astronomers'}, {'Diameter:': '6,779 km', 'Mass:': '6.39 × 10^23 kg', 'Moons:': '2', 'Distance from Sun:': '227,943,824 km', 'Length of Year:': '687 Earth days', 'Temperature:': '-153 to 20 °C'}, {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & Deimos)', 'Orbit Distance:': '227,943,824 km (1.38 AU)', 'Orbit Period:': '687 days (1.9 years)', 'Surface Temperature:': '-87 to -5 °C', 'First Record:': '2nd millennium BC', 'Recorded By:': 'Egyptian astronomers'}]


In [6]:
def scrape_nasa():
    browser = init_browser()
    
    url = "https://mars.nasa.gov/news/"
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    grid = soup.find('div',class_ = 'grid_layout')
    
    article = grid.find_all('div', class_ = 'list_text')[0]
    
    news_title = article.find('div',class_ = 'content_title').get_text()
    
    news_p = article.find('div',class_ = 'article_teaser_body').get_text()
    
    return news_title, news_p

In [7]:
news_title,news_p = scrape_nasa()
print(news_title)
print(news_p)

NASA's Perseverance Rover Goes Through Trials by Fire, Ice, Light and Sound
The agency's new Mars rover is put through a series of tests in vacuum chambers, acoustic chambers and more to get ready for the Red Planet.


In [8]:
def scrape_jpl():
    browser = init_browser()
    
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    container = soup.find('div', class_ = 'carousel_container')
    image = container.find_all('article', class_ = 'carousel_item')[0]['style']
    
    featured_image_url = image.split('url(')[1]
    
    featured_image_url = featured_image_url.strip(')')
    featured_image_url = featured_image_url.strip("'")
    featured_image_url = featured_image_url.strip(';')
    featured_image_url = featured_image_url.strip(')')
    featured_image_url = featured_image_url.strip("'")
    
    return featured_image_url

In [9]:
featured_image_url = 'https://www.jpl.nasa.gov' + scrape_jpl()
print(featured_image_url)  

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19980-1920x1200.jpg


In [82]:
def scrape_twitter():
    browser = init_browser()
    
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    time.sleep(1)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    soup = soup.find('div', {'class':'css-1dbjc4n'})
    article = soup.find_all('article')[0]
    tweets = article.find_all('span', class_ = 'css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0')
    
    for item in tweets:
        if len(item.get_text()) > 50:
            return item.get_text()

In [83]:
mars_weather = scrape_twitter()
print(mars_weather)

InSight sol 528 (2020-05-22) low -93.5ºC (-136.3ºF) high -5.3ºC (22.4ºF)
winds from the SW at 6.3 m/s (14.0 mph) gusting to 23.6 m/s (52.8 mph)
pressure at 7.10 hPa


In [ ]:
def scrape_facts():
    url = 'https://space-facts.com/mars/'
    
    facts_data = pd.read_html(url)
    
    return facts_data

In [ ]:
facts_data = scrape_facts()

In [32]:
def find_hemi(browser, link):
    url = f'https://astrogeology.usgs.gov{link}'
    
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    soup = soup.find_all('li')[0]
    image_url = soup.find('a',{'target':'_blank'})['href']
    
    return image_url

In [33]:
def scrape_hemis():
    browser = init_browser()

    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

    browser.visit(url)
    time.sleep(1)

    html = browser.html
    soup = bs(html, 'html.parser')

    container = soup.find('div',{'class':'container'})
    table = container.find('div',{'class':'collapsible results'})
    hemispheres = table.find_all('div',{'class':'description'})
    new_hemispheres = []
    for hemi in hemispheres:
        new_hemispheres.append(hemi.find('a',{'class':'itemLink product-item'})['href'])
    image_links = []
    for hemi in new_hemispheres:
        image_links.append(find_hemi(browser, hemi))
        
    return image_links

In [34]:
scrape_hemis()

['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [94]:
def scrape_hemi():
    url = 'https://www.usgs.gov/science-explorer-results?es=mars+enhanced&classification=image'

    browser = init_browser()

    browser.visit(url)
    time.sleep(1)

    html = browser.html
    soup = bs(html,'html.parser')

    picture_urls = []
    links = []

    for linker in soup.find_all('span',class_ = 'field-content'):
        link = linker.find('a')
        try:
            x = link.get('href')
            if 'https' in x:
                links.append(x)
            
        
        except:
            continue
        
    
    for link in links:
        browser.click_link_by_href(link)

        container = soup.find_all('div',class_ = 'main-container container')[0]
        container = container.find_all('div', class_ = 'row')[0]
        print(container.prettify())
        print("___________________________________________________")
        container = container.find('div',class_ = 'atom-links')
        print(container.prettify())
        print("___________________________________________________")

        image_url = container.find_all('a',class_ = 'btn btn-green')
        print(image_url)

        picture_urls.append(image_url)

        browser.back()

    return picture_urls
        



In [95]:
picture_urls = scrape_hemi()

<div class="row">
 <div class="col-sm-12">
  <h1 class="page-header">
   Science Explorer
  </h1>
  <a id="main-content">
  </a>
  <div class="region region-content">
   <div class="block block-system clearfix" id="block-system-main">
    <h2>
     Exploring: Mars Enhanced
    </h2>
    <div class="sci-explorer-quicktabs">
     <div class="row">
      <div class="sci-explorer-quicktabs col-xs-12 quicktabs-wrapper quicktabs-style-nostyle sci-explorer-quicktabs">
       <ul class="quicktabs-tabs sci-explorer-list dragscroll">
        <li class="">
         <a href="/science-explorer-results?es=mars+enhanced">
          All (28)
         </a>
        </li>
        <li class="not-active">
         <a class="quicktabs-tab quicktabs-tab-view qt-science-(1)" href="/science-explorer-results?es=mars+enhanced&amp;classification=science_project">
          Science (1)
         </a>
        </li>
        <li class="not-active">
         <a class="quicktabs-tab quicktabs-tab-view qt-publications-(1

AttributeError: 'NoneType' object has no attribute 'prettify'

In [16]:
import urllib

In [17]:
url = "https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif"

In [19]:
url.split('/')[-1].split('.')[0]

'cerberus_enhanced'

In [24]:
urllib.request.urlretrieve(url,f"Resources/{url.split('/')[-1].split('.')[0]}.jpg")

('Resources/cerberus_enhanced.jpg', <http.client.HTTPMessage at 0x116c74c10>)